In [517]:
# Import the pandas package, then use the "read_csv" function to read
# the labeled training data
import pandas as pd       
train = pd.read_csv("input/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [518]:
# Import lxml.html 
import lxml.html as lh

In [519]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords # Import the stop word list
stops = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sunggon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [520]:
import re
from nltk.stem import PorterStemmer
ps = PorterStemmer()
# Create a function to work with 25,000 taining reviews
def review_to_words(raw_review):
    review_text = lh.fromstring(raw_review,"lxml").text_content()
    words = re.findall(r'\w+', review_text.lower())
    meaningful_words = [ w for w in words if not w in stops]
    return(" ".join(meaningful_words))
#    meaningful_words = [ ps.stem(w) for w in words if not w in stops]
#    return(" ".join(meaningful_words))

In [521]:

y = train["sentiment"] 

In [522]:
# Get the number of reviews based on the dataframe column size
num_reviews = train["review"].size

In [523]:
print("Cleaning and parsing the training set movie reviews ... \n")
traindata = []
for i in range(0, num_reviews):
    # If the index is evenly divisible by 1000, print a message
    if((i+1)%1000 == 0 ):
        print("Review %d of %d\n" %(i+1, num_reviews))
    traindata.append(review_to_words(train["review"][i]))

Cleaning and parsing the training set movie reviews ... 

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [524]:
# Read the test data
test = pd.read_csv("input/testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )
num_reviews = test["review"].size

In [525]:
testdata = []
for i in range(0, num_reviews):
    # If the index is evenly divisible by 1000, print a message
    if((i+1)%1000 == 0 ):
        print("Review %d of %d\n" %(i+1, num_reviews))
    testdata.append(review_to_words(test["review"][i]))

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [526]:
X_all = traindata + testdata
lentrain = len(traindata)

In [527]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=18, ngram_range=(1, 2), sublinear_tf=True)


print("fitting pipeline... ")
tfv.fit(X_all)
X_all = tfv.transform(X_all)

X_train = X_all[:lentrain]
X_test = X_all[lentrain:]

Creating the bag of words...

fitting pipeline... 


In [528]:
from sklearn.linear_model import LogisticRegression
print("==================== sklearn LogisticRegression ==============")
classifier = LogisticRegression()

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
classifier = classifier.fit(X_train, y)


==================== sklearn LogisticRegression ==============


In [529]:
# Use the random forest to make sentiment label predictions
result = classifier.predict_proba(X_test)[:,1]

In [530]:
# Copy the results to a pandas dataframe with an "id" column and
# a 'sentiment' column
output = pd.DataFrame(data={"id":test["id"],"sentiment":result})

# Use pandas to write the comma-seperated output file
output.to_csv("Bag_of_Words_model_tfidf_LogisticRegression.csv", index=False, quoting=3)